# BASE DE DATOS
-------------------------------------------

## MONGO DB
------------

Mongo DB es una base de datos no relacional. Toda la información necesaria a su instalación se puede encontrar en: [Instalación MongoDB en Ubuntu](https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-ubuntu/)

Una vez iniciado mongoDB en local podemos acceder al mismo empleando la URI local: `mongodb://localhost:27017`.

Para manejar mongoDB en python debemos de instalar la libreria pymongo y importar MongoClient.

Un ejemplo de como iniciar mongo en python:

```
from pymongo import MongoClient

# connect with the client
Mongo_URI = 'mongodb://localhost:27017'
client = MongoClient(Mongo_URI)

# connect with the database
db = client.<databasename>

# connect with a collection
collection = db.<collectionname>
```

Dentro de cada colección encontramos documetos que simbolizaría cada entrada en la base de datos. Por ejemplo, en la base de datos *Northwind* encontramos la colección *Customers*, con documentos haciendo referencia a cada uno de esos clientes.

In [9]:
from pymongo import MongoClient, collation
from bson.objectid import ObjectId
from pprint import pprint

MONGO_URI="mongodb://localhost:27017"
client = MongoClient(MONGO_URI)
db = client.Northwind
collection = db.Customers

print("\nHas accedido con éxito a la colección clientes")
print(f"El objeto colección : \n{collection}")

Server Status:
False

Has accedido con éxito a la colección clientes
El objeto colección : 
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Northwind'), 'Customers')


### BUSQUEDA EN MONGO:

Una vez hemos cargado un objeto que nos permite conectarnos con la base de datos a la colección *Customers* podemos realizar búsquedas empleando los siguientes comandos:

1. \<CollectionObject\>.find({}) : toma un objeto tipo diccionario con los criterios de búsqueda y devuelve un cursor con todos los resultados que satisfagan los criterios especificados.

1. \<CollectionObject\>.find_one({}) : similar a find pero solo devuelve el primer resultado que encuentra en la base de datos.

1. \<CollectionObject\>.aggregate([]) : recibe una colección de objetos y añadir varios filtros para buscar en varias bases de datos o con varios criterios distintos.

**¿Como construir los criterios de busqueda?**:

Ejemplo de como buscar todos los clientes de un país:

```
{
    "Country": "USA"
}
```

Ejemplo de como excluir a los clientes de un país:

```
{
    "Country": { "$ne" : "USA"}
}
```